<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from librosa.core import resample, load
from librosa.util import fix_length
from librosa import feature
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm as nb_tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import umap
import plotly.express as px
import plotly.graph_objs as go
from IPython.display import display
import ipywidgets
from ipywidgets import Output, VBox
import IPython

In [2]:
df = pd.read_csv('full.csv',index_col=0)
df = df.fillna(0)

In [3]:
df.head(10)

,filename,dir_1,class_Hat,subclass_Hat_Open,subclass_Hat_Close,subclass_Hat_Foot,class_Bongo,class_Cymbal,subclass_Cymbal_Crash,subclass_Cymbal_Ride,...,subclass_Shaken_Maracas,class_Cajon,class_clave,dir_2,subclass_Snare_Off,subclass_Snare_On,subclass_Snare_Side,subclass_Snare_Flam,subclass_Snare_Brush,subclass_Cymbal_Trash
path,,,,,,,,,,,,,,,,,,,,,
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 OPHAT2.wav,MaxV - RX11 OPHAT2.wav,Yamaha RX-11,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 BD3.wav,MaxV - RX11 BD3.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 BD2.wav,MaxV - RX11 BD2.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 CLHAT2.wav,MaxV - RX11 CLHAT2.wav,Yamaha RX-11,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 RIDE.wav,MaxV - RX11 RIDE.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 CLAP2.wav,MaxV - RX11 CLAP2.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 MED SN.wav,MaxV - RX11 MED SN.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 CLAP1.wav,MaxV - RX11 CLAP1.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
./Samples/200-drum-machines/Yamaha RX-11/MaxV - RX11 RIM2.wav,MaxV - RX11 RIM2.wav,Yamaha RX-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# features = pd.DataFrame()

In [88]:
SR=22050
rms_list = []
rms_cent_list = []
zero_cross_list = []
spect_cent_list = []
spect_ro_list = []
spect_cont_list = []
mfcc_list = []
index_list = []
#data = data.sample(frac=0.2)
for index, file in tqdm(zip(df.index, df['path']), total=len(df.index), position=0, leave=True):
    try:
        sound, _ = load(file, sr=SR)
    except:
        continue
    
    index_list.append(index)
    
    rms = feature.rms(sound)[0]
     #normalize by rms of attack frame
    rms_centroid = (rms * np.arange(len(rms))).sum() / rms.sum()
#     features.loc[index, 'rms_centroid'] = rms_centroid
    rms_cent_list.append(rms_centroid)
    
    sound = sound/rms.max()
    sound = fix_length(sound, 1024*(SR//1024))
    
    zero_cr = feature.zero_crossing_rate(sound)[0]
#     columns = ['zero_cr_' + str(x) for x in range(zero_cr.shape[0])]
#     for col, value in zip(columns, zero_cr):
#         features.loc[index, col] = value
    zero_cross_list.append(zero_cr)
        
    spect_cent = feature.spectral_centroid(sound, sr=SR)[0]
#     columns = ['spect_cent_' + str(x) for x in range(spect_cent.shape[0])]
#     for col, value in zip(columns, spect_cent):
#         features.loc[index, col] = value
    spect_cent_list.append(spect_cent)
        
    spect_ro = feature.spectral_rolloff(sound, sr=SR)[0]
#     columns = ['spect_ro_' + str(x) for x in range(spect_ro.shape[0])]
#     for col, value in zip(columns, spect_ro):
#         features.loc[index, col] = value
    spect_ro_list.append(spect_ro)
        
    rms = feature.rms(sound)[0]
#     columns = ['rms_' + str(x) for x in range(rms.shape[0])]
#     for col, value in zip(columns, rms):
#         #features.loc[index, col] = value
#         features.loc[index, 'norm_'+ col] = value
    rms_list.append(rms)
    
        
    spect_cont = feature.spectral_contrast(sound, sr=SR)
#     for i in range (spect_cont.shape[0]):
#         for j in range (spect_cont.shape[1]):
#             col = 'spec_cont_' + str(i) + '_' + str(j)
#             features.loc[index, col] = spect_cont[i][j]
    spect_cont_list.append(spect_cont)
    
    mfcc = feature.mfcc(sound, sr=SR)
    mfcc_list.append(mfcc)

100%|██████████| 20869/20869 [28:08<00:00, 12.36it/s] 


In [98]:
rms = np.stack(rms_list)
rms_cent = np.stack(rms_cent_list)
zero_cross = np.stack(zero_cross_list)
spect_cent = np.stack(spect_cent_list)
spect_ro = np.stack(spect_ro_list)
spect_cont = np.stack(spect_cont_list)
mfcc = np.stack(mfcc_list)
index = np.stack(index_list)

np.save('feature_rms.npy', rms)
np.save('feature_rms_centroid.npy', rms_cent)
np.save('feature_zero_crossing_rate.npy', zero_cross)
np.save('feature_spectral_centroid.npy', spect_cent)
np.save('feature_spectral_rolloff.npy', spect_ro)
np.save('feature_spectral_contour.npy', spect_cont)
np.save('feature_mfcc.npy', mfcc)
np.save('index.npy', index)

In [5]:
rms = np.load('feature_rms.npy')
rms_cent = np.load('feature_rms_centroid.npy')
zero_cross = np.load('feature_zero_crossing_rate.npy')
spect_cent = np.load('feature_spectral_centroid.npy')
spect_ro = np.load('feature_spectral_rolloff.npy')
spect_cont = np.load('feature_spectral_contour.npy')
mfcc = np.load('feature_mfcc.npy')
index = np.load('index.npy')

In [6]:
index.shape

(20869,)

In [7]:
size = index.shape[0]
features = np.hstack([
    rms.reshape(size,-1),
    rms_cent.reshape(size,-1),
    zero_cross.reshape(size,-1),
    spect_cent.reshape(size,-1),
    spect_ro.reshape(size,-1),
    spect_cont.reshape(size,-1),
                  ])
features.shape

(20869, 474)

In [8]:
df = df.loc[index]

KeyError: "None of [Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,\n                9,\n            ...\n            20860, 20861, 20862, 20863, 20864, 20865, 20866, 20867, 20868,\n            20869],\n           dtype='int64', name='path', length=20869)] are in the [index]"

In [6]:
features.to_csv('features.csv')

In [ ]:
features = pd.read_csv('features.csv')

In [ ]:
df.columns

In [ ]:
classes = [c for c in df.columns if c.startswith('class_')]
cymb_classes = [c for c in df.columns if c.startswith('subclass_Cymbal')]
shak_classes = [c for c in df.columns if c.startswith('subclass_Shaken')]
hat_classes = [c for c in df.columns if c.startswith('subclass_Hat')]

all_classes = classes + cymb_classes + shak_classes + hat_classes

In [ ]:
cymb_classes

In [ ]:
class_df = pd.DataFrame()

class_df['class'] = df[all_classes].idxmax(axis=1)

mask = df[all_classes].sum(axis=1) == 0
class_df.loc[mask, 'class'] = 'Error'

cymbales = (df['class_Cymbal'] == 1) & (df[cymb_classes].sum(axis=1) > 0)
class_df.loc[cymbales, 'class'] = df.loc[cymbales, cymb_classes].idxmax(axis=1)

shaken = (df['class_Shaken'] == 1) & (df[shak_classes].sum(axis=1) > 0)
class_df.loc[shaken, 'class'] = df.loc[shaken, shak_classes].idxmax(axis=1)

hats = (df['class_Hat'] == 1) & (df[hat_classes].sum(axis=1) > 0)
class_df.loc[hats, 'class'] = df.loc[hats, hat_classes].idxmax(axis=1)
# class_df['class'] = df[hat_classes + ['class_Hat']].idxmax(axis=1)

In [ ]:
df.loc[shaken, shak_classes].idxmax(axis=1)

In [ ]:
class_df[class_df['class'].str.startswith('subclass_')]

In [ ]:
def dysplay_embedding(x, y, classes, files):
    embedding_vae_df = pd.DataFrame({
        'x': x,
        'y': y,
        'class': classes,
        'file': files    
                         })
    
    scatter = px.scatter(
        embedding_vae_df,
        x='x',
        y='y',
        color='class',
        hover_data=['file']
    )
    figure = go.FigureWidget(scatter)

    out = Output()

    @out.capture(clear_output=False, wait=True)
    def play_sound(trace, points, selector):
        if len(points.point_inds) != 1:
           return
        #print(points)
        out.clear_output()
        path = trace.customdata[points.point_inds[0]][0]
        #print(path)
        sound, sr = load(path)
        IPython.display.display(IPython.display.Audio(sound, rate=sr, autoplay=True))

    for trace in figure.data:
        trace.on_click(play_sound)


    box = VBox([figure, out])
    display(box)

In [ ]:
# from sklearn.preprocessing import StandardScaler 
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)

In [18]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(features_scaled)

NameError: name 'features_scaled' is not defined

In [ ]:
class_df = class_df.loc[index]
class_df.shape

In [ ]:
filename = df.loc[index,'path']

In [ ]:
classes_onehot_df = df[all_classes]
classes_onehot_df = classes_onehot_df.loc[index]
nb_classes =  classes_onehot_df.shape[1]
nb_features = features.shape[1]

In [21]:
dysplay_embedding(
    embedding[:, 0],
    embedding[:, 1],
    class_df['class'].values,
    filename
                 )

NameError: name 'embedding' is not defined

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
# rms_cent = rms_cent.reshape(-1, 1)
# rms = rms.reshape(-1,rms.shape[1], 1)
features_input = dict(
    rms=rms,
    rms_centroid=rms_cent,
    zero_crossing=zero_cross,
    spectral_centroid=spect_cent,
    spectral_rolloff=spect_ro,
    spectral_contour=spect_cont,
    mfcc=mfcc
)
for key, feature in list(features_input.items()):
    if feature.shape[-1] !=1:
        features_input[key] = np.expand_dims(feature, axis=feature.ndim)

In [ ]:
scaler = {}
features_scaled = {}
for key, feature in list(features_input.items()):
    scaler[key] = {}
    scaler[key]['max'] = max(feature.max(), -feature.min())
    features_scaled[key] = feature/scaler[key]['max']
#     scaler[key]['mean'] = feature.mean()
# #     scaler[key]['mean'] = feature.mean(axis=0)
#     f = feature-scaler[key]['mean']
#     scaler[key]['std'] = f.std()
# #     scaler[key]['std'] = f.std(axis=0)
#     features_scaled[key] = f/scaler[key]['std']

In [ ]:
for key, feature in list(features_scaled.items()):
    print(key, feature.min(), feature.max())

In [ ]:
for key, value in features_scaled.items():
    print(f"{key} shape: {value.shape[1:]}")

In [ ]:
latent_dim=2
def build_encoder():
    rms_input = keras.Input(shape=features_input['rms'].shape[1:])
    rms_cent_input = keras.Input(shape=features_input['rms_centroid'].shape[1:])
    zero_cross_input = keras.Input(shape=features_input['zero_crossing'].shape[1:])
    spect_cent_input = keras.Input(shape=features_input['spectral_centroid'].shape[1:])
    spect_ro_input = keras.Input(shape=features_input['spectral_rolloff'].shape[1:])
    spect_cont_input = keras.Input(shape=features_input['spectral_contour'].shape[1:])
    mfcc_input = keras.Input(shape=features_input['mfcc'].shape[1:])

    x=rms_input
    for filters in [4,8,16]:
        x = layers.Conv1D(filters,3,activation='relu',padding='same')(x)
        x = layers.MaxPooling1D(2)(x)
    rms_out = layers.Flatten()(x)

    x=zero_cross_input
    for filters in [4,8,16]:
        x = layers.Conv1D(filters,3,activation='relu',padding='same')(x)
        x = layers.MaxPooling1D(2)(x)
    zero_cross_out = layers.Flatten()(x)

    x=spect_cent_input
    for filters in [4,8,16]:
        x = layers.Conv1D(filters,3,activation='relu',padding='same')(x)
        x = layers.MaxPooling1D(2)(x)
    spect_cent_out = layers.Flatten()(x)

    x=spect_ro_input
    for filters in [4,8,16]:
        x = layers.Conv1D(filters,3,activation='relu',padding='same')(x)
        x = layers.MaxPooling1D(2)(x)
    spect_ro_out = layers.Flatten()(x)

    x=mfcc_input
    for filters in [8,16,32]:
        x = layers.Conv2D(filters, (3,3), activation='tanh',padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
    x = layers.MaxPooling2D((2,1))(x)
    mfcc_out = layers.Flatten()(x)

    x=spect_cont_input
    for filters in [4,8]:
        x = layers.Conv2D(filters, (3,3), activation='relu',padding='same')(x)
        x = layers.MaxPooling2D((2,2))(x)
    x = layers.Conv2D(16, (3,3), activation='relu',padding='same')(x)
    x = layers.MaxPooling2D((1,2))(x)
    spect_cont_out = layers.Flatten()(x)

    concat = layers.Concatenate()((
        rms_cent_input, 
        rms_out, zero_cross_out, 
        spect_cent_out,
        spect_ro_out, 
        mfcc_out, 
        spect_cont_out
    ))

    x= concat
    for size in [32, 16, 8]:
        x = layers.Dense(size, activation='relu')(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])

    inputs = dict(
        rms=rms_input,
        rms_centroid=rms_cent_input,
        zero_crossing=zero_cross_input,
        spectral_centroid=spect_cent_input,
        spectral_rolloff=spect_ro_input,
        spectral_contour=spect_cont_input,
        mfcc=mfcc_input,
    )
    outputs =  (z_mean, z_log_var, z)

    encoder = keras.Model(inputs, outputs, name="encoder")
    return encoder

In [ ]:
def build_decoder():
    latent_inputs = keras.Input(shape=(latent_dim,))

    x = layers.Dense(1 * 6 * 16, activation="relu")(latent_inputs)
    x = layers.Reshape((1, 6, 16))(x)
    x = layers.Conv2DTranspose(32, 3, activation="relu", strides=(2,2), padding="same")(x)
    x = layers.Conv2DTranspose(16, 3, activation="relu", strides=(2,2), padding="same")(x)
    x = layers.Conv2DTranspose(1, 3, activation="sigmoid", strides=(2,2), padding="same")(x)
    x = layers.Cropping2D(cropping=((0,1),(2,3)))(x)
    spect_cont_out = x

    x = layers.Dense(16, activation="relu")(latent_inputs)
    x = layers.Dense(3 * 6 * 16, activation="relu")(x)
    x = layers.Reshape((3, 6, 16))(x)
    x = layers.Conv2DTranspose(32, 3, activation="relu", strides=(2,2), padding="same")(x)
    x = layers.Conv2DTranspose(16, 3, activation="relu", strides=(2,2), padding="same")(x)
    x = layers.Conv2DTranspose(1, 3, activation="tanh", strides=(2,2), padding="same")(x)
    x = layers.Cropping2D(cropping=((2,2),(2,3)))(x)
    mfcc_out = x

    # x = layers.Dense(16, activation="relu")(latent_inputs)
    # x = layers.Dense(32, activation="relu")(x)
    # x = layers.Dense(64, activation="relu")(x)
    # x = layers.Dense(20*43, activation="relu")(x)
    # mfcc_out = layers.Reshape((20, 43, 1))(x)

    x = layers.Dense(6 * 16, activation="relu")(latent_inputs)
    x = layers.Reshape((6, 16))(x)
    x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(16, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(1, 3, activation="sigmoid", strides=2, padding="same")(x)
    x = layers.Cropping1D(cropping=(2,3))(x)
    rms_out = x

    x = layers.Dense(6 * 16, activation="relu")(latent_inputs)
    x = layers.Reshape((6, 16))(x)
    x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(16, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(1, 3, activation="sigmoid", strides=2, padding="same")(x)
    x = layers.Cropping1D(cropping=(2,3))(x)
    zero_cross_out = x

    x = layers.Dense(6 * 16, activation="relu")(latent_inputs)
    x = layers.Reshape((6, 16))(x)
    x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(16, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(1, 3, activation="sigmoid", strides=2, padding="same")(x)
    x = layers.Cropping1D(cropping=(2,3))(x)
    spect_cent_out = x

    x = layers.Dense(6 * 16, activation="relu")(latent_inputs)
    x = layers.Reshape((6, 16))(x)
    x = layers.Conv1DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(16, 3, activation="relu", strides=2, padding="same")(x)
    x = layers.Conv1DTranspose(1, 3, activation="sigmoid", strides=2, padding="same")(x)
    x = layers.Cropping1D(cropping=(2,3))(x)
    spect_ro_out = x

    for size in (4,8,16):
        x = layers.Dense(size, activation="relu")(latent_inputs)
    rms_cent_out = layers.Dense(1, activation="sigmoid")(x)

#     x = layers.Lambda(lambda x: tf.atan2(*tf.split(x,2, axis=-1)))(latent_inputs)
    x = layers.Dense(8, activation="sigmoid")(latent_inputs)
    class_out = layers.Dense(nb_classes, activation="sigmoid")(x)

    outputs = dict(
        rms=rms_out,
        rms_centroid=rms_cent_out,
        zero_crossing=zero_cross_out,
        spectral_centroid=spect_cent_out,
        spectral_rolloff=spect_ro_out,
        spectral_contour=spect_cont_out,
        mfcc=mfcc_out,
        classes=class_out
    )
    decoder_outputs = x
    #decoder_outputs = layers.Dense(input_dim, activation="relu")(x)
    decoder = keras.Model(latent_inputs, outputs, name="decoder")
    return decoder

In [ ]:
build_decoder().summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            
            rms_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data['rms'], reconstruction['rms'])
            )
            rms_loss *= 43
            zero_crossing_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data['zero_crossing'], reconstruction['zero_crossing'])
            )
            zero_crossing_loss *= 43
            spectral_centroid_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data['spectral_centroid'], reconstruction['spectral_centroid'])
            )
            spectral_centroid_loss *= 43
            spectral_rolloff_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data['spectral_rolloff'], reconstruction['spectral_rolloff'])
            )
            spectral_rolloff_loss *= 43
            mfcc_loss = tf.reduce_mean(
                keras.losses.MSE(data['mfcc'], reconstruction['mfcc'])
            )
            mfcc_loss *= 20*43
            spectral_contour_loss = tf.reduce_mean(
                keras.losses.MSE(data['spectral_contour'], reconstruction['spectral_contour'])
            )
            spectral_contour_loss *= 7*43
            rms_cent_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data['rms_centroid'], reconstruction['rms_centroid'])
            )
            
            class_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data['classes'], reconstruction['classes'])
            )
            class_loss *= nb_classes

            reconstruction_loss = rms_loss + mfcc_loss + rms_cent_loss + zero_crossing_loss \
                                + spectral_centroid_loss + spectral_rolloff_loss+ spectral_contour_loss
#             reconstruction_loss = mfcc_loss
            reconstruction_loss += class_loss
    
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
            "mfcc_loss": mfcc_loss
        }

In [ ]:
features_scaled['classes'] = classes_onehot_df.values
encoder = build_encoder()
decoder = build_decoder()
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
vae.fit(features_scaled, epochs=20, batch_size=128)

In [ ]:
latent_dim = 2
input_dim = features.shape[1]

encoder_inputs = keras.Input(shape=(input_dim))
x = layers.Dense(64, activation="relu")(encoder_inputs)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var",
                         bias_initializer='zeros', kernel_initializer='zeros',
                         )(x)
                         #activity_regularizer='l1')(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
#x = layers.BatchNormalization()(latent_inputs)
x = layers.Dense(32, activation="relu")(latent_inputs)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
decoder_outputs = layers.Dense(input_dim, activation="relu")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.MSE(data, reconstruction)
            )
            reconstruction_loss *= features.shape[1]
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

In [ ]:

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
vae.fit(features_scaled, epochs=50, batch_size=128)

In [ ]:
features_vae, _, _  = encoder.predict(features_scaled)

In [ ]:
features_vae.shape, class_df['class'].shape, filename.shape

In [ ]:
features_vae_df = pd.DataFrame(features_vae, index=index)

In [ ]:
dysplay_embedding(
    features_vae_df[ 0],
    features_vae_df[1],
    class_df['class'].values,
    filename
                 )

In [ ]:
vae_reducer = umap.UMAP()
embedding_vae = vae_reducer.fit_transform(features_vae)
#embedding = features_vae

In [ ]:
dysplay_embedding(
    embedding_vae[:, 0],
    embedding_vae[:, 1],
    class_df['class'].values,
    filename
                 )

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
decoder_outputs = layers.Dense(nb_classes, activation="relu")(latent_inputs)
class_decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

class VAE_class(keras.Model):
    def __init__(self, encoder, decoder, class_decoder, **kwargs):
        super(VAE_class, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.class_decoder = class_decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        features, classes = tf.split(data, [nb_features, nb_classes], 1)
        
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(features)
            reconstruction = self.decoder(z)
            classes_recon = self.class_decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.MSE(features, reconstruction)
            )
            class_loss = tf.reduce_mean(
                keras.losses.MSE(classes, classes_recon)
            )
            class_loss = class_loss*20
            reconstruction_loss *= nb_features/10
            class_loss *= nb_classes
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss + class_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
            "class_loss": class_loss,
        }

In [ ]:
vae = VAE_class(encoder, decoder, class_decoder)
vae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
data = np.concatenate((features_scaled, classes_onehot_df.values), axis=1)
vae.fit(data, epochs=50, batch_size=128)

features_vae2, _,_  = encoder.predict(features_scaled)

In [ ]:
features_vae_df2 = pd.DataFrame(features_vae2, index=features.index)
dysplay_embedding(
    features_vae_df2[ 0],
    features_vae_df2[1],
    class_df['class'].values,
    filename
                 )

In [ ]:
dysplay_embedding(
    features_vae_df[ 0],
    features_vae_df[1],
    class_df['class'].values,
    filename
                 )